## Data search on hosts using API

In [ ]:
import requests
from pathlib import Path
import json
import geopandas as gp
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime
from datetime import timedelta

### Credentials

In [ ]:
# PROJECT CREDENTIALS

# Path, load read json with credential secrets
proj_conf_file_path = Path.cwd()/'project-config-file.json'

# Opening JSON file and load into dict
open_proj_conf_file = open(proj_conf_file_path)
project_credentials = json.load(open_proj_conf_file)

In [ ]:
# PROJECT CREDENTIALS

client_id = project_credentials['project_id']
client_secret = project_credentials['project_api_key']

### Functions

In [ ]:
# Get access token
def get_ptoken(client_id, client_secret):
    response = requests.post("https://api.up42.com/oauth/token",
                             data={"grant_type": "client_credentials"},
                             auth=(client_id, client_secret)
                            )
    
    return response.json()["access_token"]

In [ ]:
# Get all hosts

def get_list_hosts(PTOKEN):
    response = requests.get("https://api.up42.com/hosts",
                            data={"grant_type": "client_credentials"},
                            headers={"Authorization": "Bearer " + PTOKEN}
                           )
    return response.json()["data"]

# Search data through Hosts 
def search_hosts(PTOKEN, HOST_NAME):
    response = requests.post("https://api.up42.com/catalog/hosts/"+HOST_NAME+"/stac/search",
                            headers={"Authorization": "Bearer " + PTOKEN,
                                    "Content-Type" : "application/json"},
                             data=updated_payload
                           )
    return response.json()

### Available Hosts

In [ ]:
# Build our host list calling "build_host_list()" function

PTOKEN = get_ptoken(client_id, client_secret)
#PTOKEN

hosts_name = get_list_hosts(PTOKEN)
current_host_list = [((hosts_name)[i]['name']) for i in range(len(hosts_name))]

In [ ]:
current_host_list

### AOI

In [ ]:
# Manage aoi file for use in payload or results plot

# AOI file path
    aoi_file_path = Path.cwd()/'YOUR_AOI.geojson'

# Opening JSON file and load into variable aoi
aoi_file = open(aoi_file_path)
aoi = json.load(aoi_file)

# Use geopandas to read aoi file and extract aoi bounds
gpaoi = gp.read_file(aoi_file_path)
bounds = gpaoi.total_bounds

# Bbox coordinates for payload if wanted
bbox_coord = [bounds[0], bounds[1], bounds[2], bounds[3]]

# Intersects coordinate for payload if wanted
intersects_coord = aoi['features'][0]['geometry']

aoi

### Search parameters / Payload

In [ ]:
# Search parameters from file - here we essentialy set a template payload that is built properly (we can update parameters in the next code block)

# File path
file_path = Path.cwd()/'search_params/sr_og.json'

# Opening JSON file and load into dict
sr_file = open(file_path)
payload = json.load(sr_file)
payload

In [ ]:
# Search parameters - update payload query options ("Airbus","TWENTY_ONE_AT", "ESA", "CAPELLA_SPACE", "NEAR_SPACE_LABS")
# The easiest might be building the json payload seperatly, this code block does however allow you to updated
# values from the template payload read in th eprevious code block. 
# You can modify the json by commenting on or off the del or payload lines to change you json payload file.

payload["datetime"] = "2022-01-01T00:00:00Z/2022-12-31T23:59:59Z"

del payload["intersects"]
#payload["intersects"] = intersects_coord

#del payload["bbox"]
payload["bbox"] = bbox_coord

#del payload["contains"]
#payload["contains"] = intersects_coord

#del payload["limit"]
payload["limit"] = 500

#del payload["collections"]
payload["collections"] = []

#del payload["query"]["cloudCoverage"]["lte"]
payload["query"]["cloudCoverage"]["lte"] = 50

#del payload["query"]["resolution"]["lte"]
payload["query"]["resolution"]["lte"] = 10

del payload["query"]["producer"]
#payload["query"]["producer"]["in"] = []

del payload["query"]["up42:usageType"]
#payload["query"]["up42:usageType"]["in"] = []

del payload["query"]["processingLevel"]
#payload["query"]["processingLevel"]["in"] = []

del payload["query"]["deliveryTime"]
#payload["query"]["deliveryTime"]["in"] = []

updated_payload = json.dumps(payload)
updated_payload

### Single host search

In [ ]:
# Remember tokens run out after FIVE MINUTES, refresh often

PTOKEN = get_ptoken(client_id, client_secret)

In [ ]:
# Search Results as geodataframe - change the host name to search other hosts
HOST_NAME = "oneatlas"
gp_search_result = gp.GeoDataFrame.from_features(search_hosts(PTOKEN, HOST_NAME))

In [ ]:
gp_search_result

In [ ]:
# Number of scenes found
print(f"{len(gp_search_result['collection'])} {HOST_NAME} scenes found")

# Break down per collection (mostly for oneatlas)
print(gp_search_result['collection'].value_counts())

In [ ]:
#{gp_search_result['constellation']}")
result_list = [(gp_search_result['acquisitionDate'][i], gp_search_result['constellation'][i], gp_search_result['cloudCoverage'][i]) for i in range(len(gp_search_result))]
result_list

### Plot results

In [ ]:
# Plot coverage
zoom_factor = -1
fig = plt.figure(figsize=(12, 16))

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(resolution='50m', color='darkslategray', linewidth=0.25)
ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5, zorder=1)
ax.add_feature(cfeature.LAND, zorder=0)
ax.set_extent([bounds[0]+zoom_factor, bounds[2]-zoom_factor, bounds[1]+zoom_factor, bounds[3]-zoom_factor])
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

ax.add_geometries(gpaoi.geometry, crs=ccrs.PlateCarree(),
                  edgecolor='blue', facecolor='none', zorder = 2)
#ax.add_geometries(all_host_results.geometry, crs=ccrs.PlateCarree(),
#                  edgecolor='magenta', facecolor='none', zorder = 3)
ax.add_geometries(gp_search_result.geometry, crs=ccrs.PlateCarree(),
                  edgecolor='magenta', facecolor='none', zorder = 4)


plt.show()

### Meta deep dive

In [ ]:
gp_search_result.keys()

In [ ]:
gp_search_result.providerProperties[0]

If you have any questions or would like advice on other up42 things, feel free to reach out!!
Happy developing - Alex (alex.bishop@up42.com)